# **Setup**

## **Import libraries**

In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
from datetime import datetime
import pdfplumber
from typing import Dict, Optional
import requests
import json
import re

from src.pdf_processor import PDFProcessor


## **Don't forget:**

- Install Ollama: https://ollama.ai/download
- Download a model (4-7B is enough)
- Using *qwen3:8b*, remember to enable Ollama to accept external requests!
- Run: `ollama serve`

# **Get Data**

In [ ]:

# Função principal para uso
def main():
    # Configurar o diretório onde os PDFs estão armazenados
    pdf_directory = os.path.join(os.getcwd(), "..", "CBH_LN/")
    
    # Criar processador
    processor = PDFProcessor(pdf_directory)
    
    # Processar todos os arquivos
    df_atas = processor.process_all_files()

    df_atas = processor.debug_dates(df_atas)

    
    # Exibir resultados
    print(f"\nProcessamento concluído. {len(df_atas)} atas processadas.")
    print("\nPrimeiras linhas do DataFrame:")
    print(df_atas.head())
    
    # Salvar resultados
    output_file = "atas_processadas.csv"
    df_atas.to_csv(output_file, index=False, encoding='utf-8')
    print(f"\nDataFrame salvo em: {output_file}")
    
    return df_atas

# Exemplo de uso específico
if __name__ == "__main__":
    # Executar processamento
    dataframe_atas = main()


Encontrados 45 arquivos PDF para processar...
Processando: Ata_CBH_LN_2012_02_Ord.pdf
Usando LLM para extrair data de Ata_CBH_LN_2012_02_Ord.pdf...
Processando: Ata_CBH_LN_2012_03_Ord.pdf
Usando LLM para extrair data de Ata_CBH_LN_2012_03_Ord.pdf...
Processando: Ata_CBH_LN_2013_01_Ord.pdf
Usando LLM para extrair data de Ata_CBH_LN_2013_01_Ord.pdf...
Processando: Ata_CBH_LN_2013_02_Ord.pdf
Usando LLM para extrair data de Ata_CBH_LN_2013_02_Ord.pdf...
Processando: Ata_CBH_LN_2013_03_Ord.pdf
Processando: Ata_CBH_LN_2014_01_Ord.pdf
Usando LLM para extrair data de Ata_CBH_LN_2014_01_Ord.pdf...


In [ ]:
dataframe_atas.head()

,ID,Data,CBH,Tipo,Texto
1,Ata_CBH_LN_2012_03_Ord.pdf,2012-05-03,Litoral Norte,ord,COMITÊ DAS BACIAS HIDROGRÁFICAS DO LITORAL NOR...
0,Ata_CBH_LN_2012_02_Ord.pdf,2012-07-11,Litoral Norte,ord,COMITÊ DAS BACIAS HIDROGRÁFICAS DO LITORAL\nNO...
2,Ata_CBH_LN_2013_01_Ord.pdf,2013-03-13,Litoral Norte,ord,1\nCOMITÊ DAS BACIAS HIDROGRÁFICAS DO LITORAL ...
3,Ata_CBH_LN_2013_02_Ord.pdf,2013-06-04,Litoral Norte,ord,COMITÊ DAS BACIAS HIDROGRÁFICAS DO LITORAL NOR...
4,Ata_CBH_LN_2013_03_Ord.pdf,2013-12-12,Litoral Norte,ord,COMITÊ DAS BACIAS HIDROGRÁFICAS DO LITORAL NOR...


In [ ]:
dataframe_atas[dataframe_atas['ID'] == "Ata_CBH_LN_2013_03_Ord.pdf"]

,ID,Data,CBH,Tipo,Texto
4,Ata_CBH_LN_2013_03_Ord.pdf,2013-12-12,Litoral Norte,ord,COMITÊ DAS BACIAS HIDROGRÁFICAS DO LITORAL NOR...


# **Some tests**

In [104]:
print(dataframe_atas.columns)
print(dataframe_atas[dataframe_atas['ID'] == "Ata_CBH_LN_2017_04_Ord.pdf"]['Data'])

Index(['ID', 'Data', 'CBH', 'Tipo', 'Texto'], dtype='object')
17   2017-11-16
Name: Data, dtype: datetime64[ns]


In [105]:
# verificando se todos os arquivos da pasta estão no dataframe
if (len(dataframe_atas) == len([f for f in os.listdir("CBH_LN/") 
                    if f.lower().endswith('.pdf')])):
    print("Todos os arquivos foram processados e estão no DataFrame.")

Todos os arquivos foram processados e estão no DataFrame.


In [ ]:
from datetime import datetime
import re
import pandas as pd

def contains_instance(df: pd.DataFrame, filepath: str, data: str, cbh: str, tipo: str, text_snippet: str) -> bool:
    """
    Verifica se o DataFrame contém uma instância com os dados fornecidos.
    """
    if df.empty:
        return False

    # Converter data para string no formato YYYY-MM-DD
    try:
        data_str = datetime.strptime(data, "%Y-%m-%d").strftime("%Y-%m-%d")
    except Exception:
        data_str = data

    # Normalizar o texto e o snippet para comparação (remover espaços extras, quebras de linha, etc.)
    def normalize_text(text):
        if pd.isna(text):
            return ""
        # Substituir múltiplos espaços/quebras de linha por um único espaço
        text = re.sub(r'\s+', ' ', str(text))
        return text.strip()
    
    # Filtrar DataFrame com base nos critérios fornecidos
    filtered_df = df[
        (df['ID'] == filepath) &
        (df['Data'].notna()) &
        (df['Data'].astype(str) == data_str) &
        (df['CBH'] == cbh) &
        (df['Tipo'] == tipo)
    ]
    
    # Verificar se o snippet está no texto (com normalização)
    if not filtered_df.empty:
        normalized_snippet = normalize_text(text_snippet)
        text_matches = filtered_df['Texto'].apply(
            lambda x: normalize_text(x).find(normalize_text(text_snippet)) != -1
        )
        filtered_df = filtered_df[text_matches]

    if filtered_df.empty:
        print(f"Instância não encontrada: {filepath}, {data}, {cbh}, {tipo}, snippet: '{text_snippet}'")
    
    return not filtered_df.empty

test_cases = [
    {
        "filepath": "Ata_CBH_LN_2013_03_Ord.pdf",
        "data": "2013-12-12",
        "cbh": "Litoral Norte",
        "tipo": "ord",
        "text_snippet": "AESA. E também a escolha do representante para substituir o Sr. Marlindo Francili",
        "expected": True
    },
    {
        "filepath": "Ata_CBH_LN_2017_04_Ord.pdf",
        "data": "2017-11-16",
        "cbh": "Litoral Norte",
        "tipo": "ord",
        "text_snippet": "reunião anterior; 2. Informes da diretoria; 3. Apresentação sobre o Pro",
        "expected": True
    },
    {
        "filepath": "Ata_CBH_LN_2024_01_Ord.pdf",
        "data": "2024-03-20",
        "cbh": "Litoral Norte",
        "tipo": "ord",
        "text_snippet": "Gutemberg da Silva Silvino (Representante da UFPB, Campus II, Areia/PB) pelo apo",
        "expected": True
    },
    {
        "filepath": "Ata_CBH_LN_2024_01_Ord.pdf",
        "data": "2024-03-20",
        "cbh": "Litoral Norte",
        "tipo": "ord",
        "text_snippet": "uma frase aleatória que não deve existir no texto...", # frase errada
        "expected": False
    },
    {
        "filepath": "Ata_CBH_LN_2024_01_Ord.pdf",
        "data": "2024-03-21", # data errada
        "cbh": "Litoral Norte",
        "tipo": "ord",
        "text_snippet": "temberg da Silva Silvino (Representante da UFPB,",
        "expected": False
    }
]

# Executar testes
for i, test in enumerate(test_cases):
    result = contains_instance(
        dataframe_atas,
        test["filepath"],
        test["data"],
        test["cbh"],
        test["tipo"],
        test["text_snippet"]
    )
    
    if result != test["expected"]:
        print(f"Teste {i+1} falhou: esperado {test['expected']}, obteve {result}")
        
        # Verifica cada campo individualmente
        row = dataframe_atas[dataframe_atas['ID'] == test["filepath"]]
        if row.empty:
            print(f"Falha no campo 'ID': {test['filepath']}")
        else:
            if not (row['Data'].astype(str) == test["data"]).any():
                print(f"Falha no campo 'Data': esperado {test['data']}, obtido {row['Data'].values[0]}")
            if not (row['CBH'] == test["cbh"]).any():
                print(f"Falha no campo 'CBH': esperado {test['cbh']}, obtido {row['CBH'].values[0]}")
            if not (row['Tipo'] == test["tipo"]).any():
                print(f"Falha no campo 'Tipo': esperado {test['tipo']}, obtido {row['Tipo'].values[0]}")
            
            # Verifica o texto com normalização
            def normalize_text(text):
                if pd.isna(text):
                    return ""
                text = re.sub(r'\s+', ' ', str(text))
                return text.strip()
            
            normalized_snippet = normalize_text(test["text_snippet"])
            text_matches = row['Texto'].apply(
                lambda x: normalize_text(x).find(normalized_snippet) != -1
            )
            
            if not text_matches.any():
                print(f"Falha no campo 'Texto' (snippet): '{test['text_snippet']}'")
                print("Texto normalizado para comparação:")
                print(normalize_text(test["text_snippet"]))
                print("Texto real normalizado:")
                print(normalize_text(row['Texto'].values[0]))

Instância não encontrada: Ata_CBH_LN_2024_01_Ord.pdf, 2024-03-20, Litoral Norte, ord, snippet: 'uma frase aleatória que não deve existir no texto...'
Instância não encontrada: Ata_CBH_LN_2024_01_Ord.pdf, 2024-03-21, Litoral Norte, ord, snippet: 'temberg da Silva Silvino (Representante da UFPB,'


In [ ]:
import csv
# Salvar o DataFrame em CSV com formatação adequada
dataframe_atas.to_csv('atas_processadas_final.csv', 
                     index=False, 
                     encoding='utf-8-sig', 
                     quoting=csv.QUOTE_ALL,
                     escapechar='\\')